In [9]:
import pandas as pd
import os
cwdir = os.getcwd()
cwdir = cwdir + '/'
pd.set_option('display.precision',15)
df_npa = pd.read_csv(cwdir+'DupsData/NPA_Forstage2.csv')
df_master = pd.read_csv(cwdir+'Master/Master.csv')
#df_npa.set_index('Unnamed: 0', inplace=True)
df_master['Phone'] = df_master['Phone'].astype(str)          
df_master['Phone'] = df_master['Phone'].str.replace('.0', ' ')
df_master['Practice Phone'] = df_master['Practice Phone'].astype(str)          
df_master['Practice Phone'] = df_master['Practice Phone'].str.replace('.0', ' ')
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].astype(str)          
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].str.replace('.0', ' ')
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].astype(str)          
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].str.replace('.0', ' ')

directmatch =  df_npa.index.tolist()
df_npa.drop(['check1'],axis=1)
df_npa.drop(['check2'],axis=1)


cols_practice_master = ['Practice Street Address 1', 'Practice Street Address 2','Practice City']
df_master['check1'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)
cols_practice_master = ['Shipping Address Line 1', 'Shipping Address Line 2', 'Shipping Address Line 3', 'Shipping City']
df_master['check2'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)

cols_practice_npa = ['Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name']
df_npa['check1'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)
cols_practice_npa = ['Provider First Line Business Mailing Address','Provider Second Line Business Mailing Address','Provider Business Mailing Address City Name']
df_npa['check2'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)


df_master['flag'] = None
df_master['flag1'] = None
index_r1 = []
master_id_l1 = []
pon = []
df_pp = pd.DataFrame()
address = df_master['check1'].tolist()
df_master['PP Old'] = df_master['Practice Phone']
for item in directmatch:
    a=df_npa.loc[df_npa.index == item, 'check1'].iloc[0]
    if a in address:
        b=df_master.loc[df_master['check1'] == a, 'Practice Phone'].values
        if len(b)>0:
            #master_id=df_master.loc[df_master['Practice Phone'] == a, 'Master/Company ID'].values
            indexpp = df_master['check1'][df_master['check1'] == a].index.tolist()
            c=df_npa.loc[df_npa.index == item, 'Provider Business Practice Location Address Telephone Number'].iloc[0]
            d=df_npa.loc[df_npa.index == item, 'Provider Other Organization Name'].values
            pn=df_master.loc[df_master['check1'] == a, 'Practice Name'].values
            if b[0]==c:
                master_id_l1.append(item)
            else:
                  
                df_master.loc[indexpp[0],'Practice Phone'] = c
                df_master.loc[indexpp[0],'flag'] = 'PP-NA'
                df_pp = df_pp.append([df_master[df_master.index==indexpp[0]]])
                index_r1.append(item)
                if len(d)>0 & len(pn)>=0:
                    if d[0] != pn[0]:
                        df_pp.loc[indexpp[0],'Practice Name'] = d[0]
                        
                        df_pp.loc[indexpp[0],'flag1'] = 'PN-NA'
                        pon.append(item)
            

df_pp.shape

df_pp = df_pp.drop(['check1'], axis = 1)
df_pp = df_pp.drop(['check2'], axis = 1) 
df_pp.to_csv(cwdir+'DupsData/straightPP-NPupdate1.csv')


df_master['flag2'] = None
df_master['flag3'] = None
index_r2 = []
master_id_l2 = []
pon2 = []
df_bp = pd.DataFrame()
address = df_master['check2'].tolist()
df_master['BP Old'] = df_master['Phone']
for item in directmatch:
    a=df_npa.loc[df_npa.index == item, 'check2'].iloc[0]
    if a in address:
        b=df_master.loc[df_master['check2'] == a, 'Phone'].values
        if len(b)>0:
            #master_id=df_master.loc[df_master['Practice Phone'] == a, 'Master/Company ID'].iloc[0]
            indexpp2 = df_master['check2'][df_master['check2'] == a].index.tolist()
            c=df_npa.loc[df_npa.index == item, 'Provider Business Mailing Address Telephone Number'].iloc[0]
            d=df_npa.loc[df_npa.index == item, 'Provider Organization Name (Legal Business Name)'].values
            pn=df_master.loc[df_master['check2'] == a, 'Account Name'].values
            if b[0]==c:
                #index_r2.append(item)
                master_id_l1.append(item)
            else:  
                df_master.loc[indexpp2[0],'Phone'] = c
                df_master.loc[indexpp2[0],'flag2'] = 'BP-NA'
                df_bp = df_bp.append([df_master[df_master.index==indexpp2[0]]])
                index_r2.append(item)
                if len(d)>0 & len(pn)>=0:
                    if d[0] != pn[0]:
                        df_master.loc[indexpp2[0],'Practice Name'] = d[0]
                        df_master.loc[indexpp2[0],'flag3'] = 'BN-NA'
                        pon2.append(item)
            
df_bp.shape


df_bp = df_bp.drop(['check1'], axis = 1)
df_bp = df_bp.drop(['check2'], axis = 1) 
df_bp.to_csv(cwdir+'DupsData/straightBP-NAupdate1.csv')

dropindex = index_r1 + index_r2
dropindex = list(set(dropindex))

df_npa = df_npa.drop(dropindex)


df_npa.to_csv(cwdir+'DupsData/NPA_ForFuzzy.csv')